# Upload the BEANS hiceas recordings to HF

#### 1. Download the recordings the way BEANS do it on their [GitHub](https://github.com/earthspecies/beans)

First we will navigate into the mounted data_birdset folder to download the temporary files from the Repo their and install wget & unzip as they are not on the university bash.

In [1]:
%cd '../../../../data_birdset/beans'
!pwd
!sudo apt install wget
!sudo apt install unzip

/workspace/data_birdset/beans
/workspace/data_birdset/beans


/workspace/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
wget is already the newest version (1.21.2-2ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unzip is already the newest version (6.0-26ubuntu3.2).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


Then we will run their script to download the metadata and recordings to have the same splits.

In [2]:
# Their script:
from plumbum import local, FG

local['mkdir']['data/hiceas']()
local['wget']['https://storage.googleapis.com/ml-bioacoustics-datasets/hiceas_1-20_minke-detection.zip', '-O', 'data/hiceas/hiceas.zip'] & FG
local['unzip']['data/hiceas/hiceas.zip', '-d', 'data/hiceas'] & FG

--2024-04-09 00:30:55--  https://storage.googleapis.com/ml-bioacoustics-datasets/hiceas_1-20_minke-detection.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.185.219, 142.250.186.91, 142.250.181.251, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.185.219|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1424628433 (1.3G) [application/zip]
Saving to: ‘data/hiceas/hiceas.zip’

     0K .......... .......... .......... .......... ..........  0%  277K 83m47s
    50K .......... .......... .......... .......... ..........  0%  421K 69m25s
   100K .......... .......... .......... .......... ..........  0% 1.04M 53m31s
   150K .......... .......... .......... .......... ..........  0% 1.05M 45m31s
   200K .......... .......... .......... .......... ..........  0% 1.28M 39m56s
   250K .......... .......... .......... .......... ..........  0% 1.81M 35m21s
   300K .......... .......... .......... .......... ..........  0% 

Archive:  data/hiceas/hiceas.zip
  inflating: data/hiceas/test.jsonl  
  inflating: data/hiceas/train-low.jsonl  
  inflating: data/hiceas/train.jsonl  
  inflating: data/hiceas/valid.jsonl  
  inflating: data/hiceas/wav/1705_20170827_175422_524.wav  
  inflating: data/hiceas/wav/1705_20170827_182122_524.wav  
  inflating: data/hiceas/wav/1705_20170827_183322_524.wav  
  inflating: data/hiceas/wav/1705_20170827_184922_524.wav  
  inflating: data/hiceas/wav/1705_20170827_185622_524.wav  
  inflating: data/hiceas/wav/1705_20170827_192435_544.wav  
  inflating: data/hiceas/wav/1705_20170827_192735_544.wav  
  inflating: data/hiceas/wav/1705_20170827_193835_544.wav  
  inflating: data/hiceas/wav/1705_20170827_203335_544.wav  
  inflating: data/hiceas/wav/1705_20170827_204635_544.wav  
  inflating: data/hiceas/wav/1705_20170827_204935_544.wav  
  inflating: data/hiceas/wav/1705_20170827_212335_544.wav  
  inflating: data/hiceas/wav/1705_20170827_215335_544.wav  
  inflating: data/hiceas/wav

#### 2. Convert to HF format

In [4]:
from datasets import Dataset, Audio
import pandas as pd
import jsonlines

def load_dataset(split_name):
    with jsonlines.open(f'data/hiceas/{split_name}.jsonl', 'r') as reader:
        data = list(reader)
    df = pd.DataFrame(data)
    dataset = Dataset.from_pandas(df)
    dataset = dataset.cast_column('path', Audio())
    return dataset

splits = ['train', 'train-low', 'valid', 'test']
datasets = {split: load_dataset(split) for split in splits}
datasets['train_low'] = datasets.pop('train-low') # Rename split from train-low to train_low as HF does not accept -
for split, dataset in datasets.items():
    print(dataset[0])



{'path': {'path': 'data/hiceas/wav/1705_20170827_175422_524.wav', 'array': array([ 0.08587646,  0.17553711,  0.15435791, ..., -0.06161499,
       -0.0559082 , -0.06008911]), 'sampling_rate': 22050}, 'length': 60.0, 'annotations': []}
{'path': {'path': 'data/hiceas/wav/1705_20171103_040235_105.wav', 'array': array([-0.00219727, -0.00439453, -0.00292969, ..., -0.00845337,
       -0.00805664, -0.0085144 ]), 'sampling_rate': 22050}, 'length': 60.0, 'annotations': [{'ed': 19, 'label': 1, 'st': 0}]}
{'path': {'path': 'data/hiceas/wav/1705_20171118_033936_019.wav', 'array': array([-0.00375366, -0.00778198, -0.00665283, ...,  0.00872803,
        0.00799561,  0.00991821]), 'sampling_rate': 22050}, 'length': 60.0, 'annotations': [{'ed': 7, 'label': 1, 'st': 0}]}
{'path': {'path': 'data/hiceas/wav/1705_20170827_175422_524.wav', 'array': array([ 0.08587646,  0.17553711,  0.15435791, ..., -0.06161499,
       -0.0559082 , -0.06008911]), 'sampling_rate': 22050}, 'length': 60.0, 'annotations': []}


#### 3. Upload the datasets to HF

In [8]:
for split, dataset in datasets.items():
    print(split)
    dataset.push_to_hub('DBD-research-group/beans_hiceas', split=split)

train


Uploading the dataset shards:   0%|          | 0/3 [00:00<?, ?it/s]

Map:   0%|          | 0/136 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Map:   0%|          | 0/136 [00:00<?, ? examples/s]

...

train_low


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/82 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

ValueError: Features of the new split don't match the features of the existing splits on the hub: {'path': Audio(sampling_rate=None, mono=True, decode=True, id=None), 'length': Value(dtype='float64', id=None), 'annotations': Sequence(feature=Value(dtype='null', id=None), length=-1, id=None)} != {'path': Audio(sampling_rate=None, mono=True, decode=True, id=None), 'length': Value(dtype='float64', id=None), 'annotations': [{'ed': Value(dtype='int64', id=None), 'label': Value(dtype='int64', id=None), 'st': Value(dtype='int64', id=None)}]}

---
# Download the dataset from HF

Download all splits from the hub. Even when specifying a specific split it still downloads everything! Use `streaming=True` and `cache_dir='...'` for shorter loading times.

In [5]:
from datasets import load_dataset, DatasetDict

#dataset = load_dataset(path='DBD-research-group/beans_hiceas', split='test')
dataset: DatasetDict = load_dataset(name='default', path='DBD-research-group/beans_hiceas')

Now you can interact with the dataset. 

In [8]:
# print number of samples and number of distinct classes
print(f"Number of samples: {len(dataset['train'])}")
dataset['train'][0]
# path -> Array contains the Audio data

Number of samples: 702


{'path': {'path': '2015-09-04_08-04-59_unit03.000.wav',
  'array': array([-0.00088501, -0.01126099, -0.00408936, ..., -0.00488281,
         -0.00430298,  0.00384521]),
  'sampling_rate': 16000},
 'length': 60.0,
 'annotations': [{'ed': 30.857, 'label': 'OVEN', 'st': 30.707},
  {'ed': 45.624, 'label': 'SWTH', 'st': 45.474},
  {'ed': 59.246, 'label': 'OVEN', 'st': 59.096}]}